In [49]:
from sradio.io.shower.zhaires_master import ZhairesMaster
import re
import numpy as np
import subprocess

Function to find the event number associated to the simulation.

Input : input directory of the simulation

Note : the event number is the last sequence of numbers in the name of the directory

In [50]:
def EventNumber(input_directory):
    last_numbers = re.search(r'\d+/$', input_directory)
    if last_numbers:
        eventnumber = last_numbers.group(0)
        eventnumber = [eventnumber.rstrip('/')]
        return eventnumber
    else:
        return None

Read simulation files and extract relevant parameters for reconstruction.

A filter is applied on the frequencry range [50, 200] MHz.

Two conditions are applied to keep the traces : at least 5 antennas have to be triggered and we keep only the traces where the signal is > 150 $\mu V/m$

Inputs : -simulation directory (that contains the hdf5 file and the other files)

-antenna threshold and amplitude threshold

Outputs : -a numpy array AntennaParameters : [antenna_index, peaktime, peakamplitude, x, y, z]   

-a dictionnary ShowerParameters : {t_sample_ns, x_max:{alt, dist, x, y, z}, sl_depth_of_max, ground_altitude, vers_zhaires, primary, site:{name, lat, long}, geo_mag1:{norm}, geo_mag2:{inc, decl}, energy, shower_zenith, shower_azimuth}


In [51]:
def GetSimulationReconstructionParameters(input_directory, antennathreshold, amplitudethreshold):
    d_event = ZhairesMaster(input_directory)
    event = d_event.get_object_3dtraces()
    event.traces = event.get_traces_passband([50,200])
    idx = event.remove_traces_low_signal(amplitudethreshold)
    peaktime= event.get_tmax_vmax()[0]*1e-9 #convert from ns to s
    peakamplitude = event.get_tmax_vmax()[1]
    antennas = d_event.f_zhaires
    antennas.read_antpos_file()
    antennas.ants['idx'] = antennas.ants['idx']-1
    index =np.isin(antennas.ants['idx'], idx)
    if len(peaktime) > antennathreshold:
        antenna_x = antennas.ants[index]['x']
        antenna_y = antennas.ants[index]['y']
        antenna_z = antennas.ants[index]['z']
        antenna_parameters = np.array([idx, peaktime, peakamplitude, antenna_x, antenna_y, antenna_z]).T
        shower_parameters = d_event.get_simu_info()
    else:
        print("Not enough antennas above threshold for issuing a T2 trigger")
    return antenna_parameters, shower_parameters


Write the coord_antennas table for the GRAND reconstruction software

Inputs : -file name (coord_antennas.txt) and directory

-antenna parameters array : [antenna_index, peaktime, peakamplitude, x, y, z] 

Output : return 0 and create a text file with id, x (m), y (m), z (m)

Note : antennas ID has to start from 0 and not repeat themself -> fake antenna ID (counter starting from 0 and up to len(total antennas))

In [52]:
def WriteAntennaPositionTable(filename, directory, antenna_params):
    fake_idx = np.arange(0, len(antenna_params[:,0]))
    antenna_x = antenna_params[:,3]
    antenna_y = antenna_params[:,4]
    antenna_z = antenna_params[:,5]
    file_path = f"{directory}{filename}"
    with open(file_path, 'w') as file:
        for i in range(len(fake_idx)):
            file.write(f"{fake_idx[i]} {antenna_x[i]} {antenna_y[i]} {antenna_z[i]}\n")
    return 0



 Write the Rec_coinctable table for the GRAND reconstruction software

 Inputs: -file name (Rec_coinctable.txt) and directory

-id_event 

-antenna parameter array: [antenna_index, peaktime, peakamplitude, x, y, z] 

Output : return 0 and write a create a textfile with antenna_id, event_id, peaktime (s), peakamplitude ($\mu V/m$)

Note : antennas ID has to start from 0 and not repeat themself -> fake antenna ID (counter starting from 0 and up to len(total antennas))


 

In [53]:
def WriteReconsTable(filename, directory, idevent, antenna_params):
    fake_idx = np.arange(0, len(antenna_params[:,0]))
    peaktime = antenna_params[:,1]
    peakamplitude = antenna_params[:,2]
    event_number = idevent*len(peaktime)
    file_path = f"{directory}{filename}"
    with open(file_path, 'w') as file:
        for i in range(len(fake_idx)):
            file.write(f"{fake_idx[i]} {event_number[i]} {peaktime[i]} {peakamplitude[i]}\n")
    return 0

In [54]:
def main():
    simu_directory = '/Users/mguelfan/Documents/GRAND/ADF_DC2/simus/Stshp_DH_S23d_Proton_3.98_77.4_45.0_1/'
    directory = "/Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/"
    filename_position = "coord_antennas.txt"
    filename_arrivaltime = 'Rec_coinctable.txt'
    antennathreshold = 5
    amplitudethreshold = 110    
    print(GetSimulationReconstructionParameters(simu_directory, antennathreshold, amplitudethreshold)[1])
    antenna_params = GetSimulationReconstructionParameters(simu_directory, antennathreshold, amplitudethreshold)[0]
    idevent = EventNumber(simu_directory)
    WriteAntennaPositionTable(filename_position, directory, antenna_params) 
    WriteReconsTable(filename_arrivaltime, directory, idevent, antenna_params)
    #print(GetSimulationReconstructionParameters(simu_directory, antennathreshold, amplitudethreshold)[1])

if __name__ == "__main__":
    main()
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 164, 165, 167, 169, 175]
['A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 

In [27]:
import math 

def WriteValueAngleSim(simu_directory, directory, file_simuangle):
    d_event = ZhairesMaster(simu_directory)
    shower_parameters = d_event.get_simu_info()
    #event_number = idevent
    zenith_angle = shower_parameters.get('shower_zenith')
    azimuth_angle = shower_parameters.get('shower_azimuth')
    split_simu_directory = simu_directory.split('/')
    simu_name = split_simu_directory[-2]
    file_path = f"{directory}{file_simuangle}"
    with open(file_path, 'a') as file:
        if azimuth_angle < 180:
            result = 180 + azimuth_angle
            print('bonjour')
            file.write(f"{simu_name} {zenith_angle:.1f} {azimuth_angle:.1f} {180-zenith_angle:.1f} {result:.1f}\n")
        else:
            angle_degreesbis = 180 + azimuth_angle
            angle_radians = math.radians(angle_degreesbis)
            result = angle_radians % (2*math.pi)
            result = np.rad2deg(result)
            file.write(f"{simu_name} {zenith_angle:.1f} {azimuth_angle:.1f} {180-zenith_angle:.1f} {result:.1f}\n")
    return 0
        



    

In [170]:
def main():
    simu_directory = '/Users/mguelfan/Documents/GRAND/ADF_DC2/simus/Stshp_DH_S23d_Proton_3.98_77.4_90.0_1/'
    directory = "/Users/mguelfan/Documents/GRAND/ADF_DC2/output_recons_ADF/"
    file_simuangle = "angle_value_simulations-MCMC.txt"
    idevent = EventNumber(simu_directory)
    WriteValueAngleSim(simu_directory, directory, file_simuangle)

if __name__ == "__main__":
    main()

    

bonjour
